## Set Up

In [ ]:
!pip install git+https://github.com/google-research/bigbird.git -q

     |████████████████████████████████| 1.2MB 13.4MB/s 
     |████████████████████████████████| 3.4MB 53.7MB/s 
     |████████████████████████████████| 1.5MB 54.2MB/s 
     |████████████████████████████████| 3.8MB 54.5MB/s 
     |████████████████████████████████| 706kB 47.9MB/s 
     |████████████████████████████████| 5.6MB 47.9MB/s 
     |████████████████████████████████| 368kB 53.1MB/s 
     |████████████████████████████████| 378kB 53.4MB/s 
     |████████████████████████████████| 194kB 53.3MB/s 
     |████████████████████████████████| 368kB 50.7MB/s 
     |████████████████████████████████| 358kB 51.2MB/s 
     |████████████████████████████████| 983kB 48.1MB/s 
     |████████████████████████████████| 655kB 47.3MB/s 
     |████████████████████████████████| 256kB 58.4MB/s 
ERROR: kfac 0.2.3 has requirement tensorflow-probability==0.8, but you'll have tensorflow-probability 0.7.0 which is incompatible.


In [ ]:
from bigbird.core import flags
from bigbird.core import modeling
from bigbird.core import utils
from bigbird.summarization import run_summarization
import tensorflow.compat.v2 as tf
import tensorflow_datasets as tfds
import tensorflow_text as tft
from tqdm import tqdm
import sys

FLAGS = flags.FLAGS
if not hasattr(FLAGS, "f"): flags.DEFINE_string("f", "", "")
FLAGS(sys.argv)

tf.enable_v2_behavior()

## Set options

In [ ]:
FLAGS.data_dir = "tfds://scientific_papers/pubmed"
FLAGS.attention_type = "block_sparse"
FLAGS.couple_encoder_decoder = True
FLAGS.max_encoder_length = 2048  # on free colab only lower memory GPU like T4 is available
FLAGS.max_decoder_length = 256
FLAGS.block_size = 64
FLAGS.learning_rate = 1e-5
FLAGS.num_train_steps = 1000
FLAGS.attention_probs_dropout_prob = 0.0
FLAGS.hidden_dropout_prob = 0.0
FLAGS.vocab_model_file = "gpt2"

In [ ]:
transformer_config = flags.as_dictionary()

## Define summarization model

In [ ]:
from tensorflow.python.ops.variable_scope import EagerVariableStore
container = EagerVariableStore()

In [ ]:
with container.as_default():
  model = modeling.TransformerModel(transformer_config)

In [ ]:
@tf.function(experimental_compile=True)
def fwd_bwd(features, labels):
  with tf.GradientTape() as g:
    (llh, logits, pred_ids), _ = model(features, target_ids=labels,
                                       training=True)
    loss = run_summarization.padded_cross_entropy_loss(
        logits, labels,
        transformer_config["label_smoothing"],
        transformer_config["vocab_size"])
  grads = g.gradient(loss, model.trainable_weights)
  return loss, llh, logits, pred_ids, grads

## Dataset pipeline

In [ ]:
train_input_fn = run_summarization.input_fn_builder(
        data_dir=FLAGS.data_dir,
        vocab_model_file=FLAGS.vocab_model_file,
        max_encoder_length=FLAGS.max_encoder_length,
        max_decoder_length=FLAGS.max_decoder_length,
        substitute_newline=FLAGS.substitute_newline,
        is_training=True)
dataset = train_input_fn({'batch_size': 2})

Dataset scientific_papers downloaded and prepared to /root/tensorflow_datasets/scientific_papers/pubmed/1.1.1. Subsequent calls will reuse this data.


/usr/local/lib/python3.7/dist-packages/bigbird/summarization/run_summarization.py:205: UserWarning: Creating resources inside a function passed to Dataset.map() is not supported. Create each resource outside the function, and capture it inside the function to use it.
  deterministic=is_training)


In [ ]:
# inspect at a few examples
for ex in dataset.take(3):
  print(ex)

(<tf.Tensor: shape=(2, 2048), dtype=int32, numpy=
array([[  382, 18450,   387, ...,  1503,  1372,   387],
       [  363,   321,   100, ...,   358,   936,   427]], dtype=int32)>, <tf.Tensor: shape=(2, 256), dtype=int32, numpy=
array([[10913, 21105, 31760,   583,  1228,   490,  3278,   456,  5972,
          430,  1793, 13926,   865,   388,   529,  2151,   321,   100,
          457, 15576,   363,  5509,   387, 49438,   458,   401,   639,
         1368,   391,   321,  9290,  7602, 33085, 43802,   458,   321,
         9290,  7602,   181,  1368, 10913,   388,  8796,   385,  2320,
          743,   113,  2805, 15331,   494,   743,   185,  2314, 17771,
         1159,   633,   118, 50035,   171,  1976,   358,   458,   401,
          639,  1368,   938,   779,   165,  1976,   409,   458,   401,
          639,  1368,   391,   633,  1258, 26615,   167,  1976,   409,
          458,   321,  9290,  7602,   181,  1368,   388, 25346, 20880,
         1852,   391, 25548,   865,   321,   100,  3686, 27056, 

## Check outputs

In [ ]:
with container.as_default():
  loss, llh, logits, pred_ids, grads = fwd_bwd(ex[0], ex[1])
print('Loss: ', loss)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer.py:1402: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`layer.updates` will be removed in a future version. '


Loss:  tf.Tensor(9.549082, shape=(), dtype=float32)


## (Optionally) Load pretrained model

In [ ]:
# For training from scratch use
# ckpt_path = 'gs://bigbird-transformer/pretrain/bigbr_base/model.ckpt-0'
# For quick check continue from trained checkpoint
ckpt_path = 'gs://bigbird-transformer/summarization/pubmed/roberta/model.ckpt-300000'
ckpt_reader = tf.compat.v1.train.NewCheckpointReader(ckpt_path)
loaded_weights = []
for v in tqdm(model.trainable_weights, position=0):
  try:
    val = ckpt_reader.get_tensor(v.name[:-2])
  except:
    val = v.numpy()
  loaded_weights.append(val)

model.set_weights(loaded_weights)

100%|██████████| 316/316 [01:20<00:00,  3.93it/s]


## Train

In [ ]:
opt = tf.keras.optimizers.Adam(FLAGS.learning_rate)
train_loss = tf.keras.metrics.Mean(name='train_loss')

for i, ex in enumerate(tqdm(dataset.take(FLAGS.num_train_steps), position=0)):
  loss, llh, logits, pred_ids, grads = fwd_bwd(ex[0], ex[1])
  opt.apply_gradients(zip(grads, model.trainable_weights))
  train_loss(loss)
  if i% 10 == 0:
    print('Loss = {} '.format(train_loss.result().numpy()))

  0%|          | 1/1000 [01:50<30:42:31, 110.66s/it]

Loss = 1.363891363143921 


  1%|          | 11/1000 [02:02<1:10:27,  4.27s/it]

Loss = 2.0001139640808105 


  2%|▏         | 21/1000 [02:14<20:59,  1.29s/it]

...

 98%|█████████▊| 981/1000 [23:36<00:25,  1.36s/it]

Loss = 1.2840265035629272 


 99%|█████████▉| 991/1000 [23:49<00:12,  1.34s/it]

Loss = 1.283961534500122 


100%|██████████| 1000/1000 [24:01<00:00,  1.44s/it]


### Print predictions

In [ ]:
tokenizer = tft.SentencepieceTokenizer(
        model=tf.io.gfile.GFile(FLAGS.vocab_model_file, "rb").read())

In [ ]:
@tf.function(experimental_compile=True)
def fwd_only(features, labels):
  (llh, logits, pred_ids), _ = model(features, target_ids=labels,
                                       training=False)
  return llh, logits, pred_ids

In [ ]:
_, _, pred_ids = fwd_only(ex[0], ex[1])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer.py:1402: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`layer.updates` will be removed in a future version. '


Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.while_loop(c, b, vars, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.while_loop(c, b, vars))


Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.while_loop(c, b, vars, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.while_loop(c, b, vars))


In [ ]:
print('Article:\n {}\n\n Predicted summary:\n {}\n\n Ground truth summary:\n {}\n\n'.format(
    tokenizer.detokenize(ex[0]),
    tokenizer.detokenize(pred_ids),
    tokenizer.detokenize(ex[1])))

Article:
 [b"the last decade has witnessed significant advances in the surgical management of cervical radiculopathy and myelopathy , to include development of motion - sparing alternatives to traditional anterior cervical diskectomy and fusion ( acdf ) . \xe2\x81\x87 the theoretical benefits of these alternatives , primarily cervical disk arthroplasty ( cda ) , include diminished contiguous level strain , preservation of motion at the affected vertebral segment , and a hypothetical decrease in the development or progression of degenerative disease processes at immediately adjacent levels.1  \xe2\x81\x87  2  \xe2\x81\x87  3 several small randomized controlled trials have suggested that cda may be associated with better neurologic outcomes , fewer revisions , and better overall success when compared with acdf.4  \xe2\x81\x87  5  \xe2\x81\x87  6  \xe2\x81\x87  7  \xe2\x81\x87  8 previous studies have found that cda provides up to an 89% rate of complete preoperative symptom relief , with

## Eval

In [ ]:
eval_input_fn = run_summarization.input_fn_builder(
        data_dir=FLAGS.data_dir,
        vocab_model_file=FLAGS.vocab_model_file,
        max_encoder_length=FLAGS.max_encoder_length,
        max_decoder_length=FLAGS.max_decoder_length,
        substitute_newline=FLAGS.substitute_newline,
        is_training=False)
eval_dataset = eval_input_fn({'batch_size': 2})

/usr/local/lib/python3.7/dist-packages/bigbird/summarization/run_summarization.py:205: UserWarning: Creating resources inside a function passed to Dataset.map() is not supported. Create each resource outside the function, and capture it inside the function to use it.
  deterministic=is_training)


In [ ]:
eval_llh = tf.keras.metrics.Mean(name='eval_llh')

for ex in tqdm(eval_dataset, position=0):
  llh, logits, pred_ids = fwd_only(ex[0], ex[1])
  eval_llh(llh)
print('Log Likelihood = {}'.format(eval_llh.result().numpy()))